In [1]:
import pandas as pd
import geopandas as gpd
import lightgbm as lgb
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer

In [2]:
faults_raw = pd.read_csv("../data/J1939Faults.csv", dtype={"EquipmentID": str, 'spn':int})
diagnostics_raw = pd.read_csv("../data/vehiclediagnosticonboarddata.csv")

In [3]:
# prepare faults
faults_drop_cols = ["actionDescription", "activeTransitionCount", "eventDescription", "ecuSource", "ecuSoftwareVersion", "ecuModel", "ecuMake", "faultValue", "MCTNumber", "LocationTimeStamp"]
faults = faults_raw.drop(columns=faults_drop_cols)
print("\n\n--------SHAPE OF FAULTS--------")
display(faults.shape)



--------SHAPE OF FAULTS--------


(1187335, 10)

In [4]:
# join diagnostics
print("--------NaNs--------")
print(diagnostics_raw.isna().sum())
n_ids = len(diagnostics_raw['Id'])
n_unique_id = diagnostics_raw['Id'].nunique()
n_un_faults = diagnostics_raw['FaultId'].nunique()
diagnostics_raw["Value"] = diagnostics_raw["Value"].replace(
    {"FALSE": False, "TRUE": True}
)

# pivot diagnostics to long format
diagnostics = diagnostics_raw.pivot(
    index="FaultId", columns="Name", values="Value"
)

print(f"\nlen(Id): {n_ids}", f"\nN unique_Id: {n_unique_id}")
print("\n--------RECORD ID vs FAULT ID--------")
print(f"n_unique FaultID: {n_un_faults}", f"\nn_unique RecordID: {faults['RecordID'].nunique()}")
joined = faults.merge(diagnostics, how = "inner", left_on='RecordID', right_on='FaultId')

--------NaNs--------
Id         0
Name       0
Value      0
FaultId    0
dtype: int64

len(Id): 12821626 
N unique_Id: 12821626

--------RECORD ID vs FAULT ID--------
n_unique FaultID: 1187335 
n_unique RecordID: 1187335


In [5]:
joined_pre_station_filter = joined
# filter out near service stations
joined_pre_station_filter = joined
print("Labeling faults near service stations...")
stations = pd.DataFrame(
    {
        "lat": [36.0666667, 35.5883333, 36.1950],
        "lon": [-86.4347222, -86.4438888, -83.174722],
    }
)
threshold_miles = 0.5
threshold_meters = threshold_miles * 1609.34
# create geodataframes with geopandas
gdf_joined = gpd.GeoDataFrame(
    joined,
    geometry=gpd.points_from_xy(joined.Latitude, joined.Longitude),
    crs="EPSG:4326",  # WGS84 coord ref sys (lat/lon)
)
gdf_stations = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations.lat, stations.lon),
    crs="EPSG:4326",
)
target_crs = "EPSG:9311"
# reproject onto new crs for better distance measurement
gdf_joined_proj = gdf_joined.to_crs(target_crs)
gdf_stations_proj = gdf_stations.to_crs(target_crs)
# create buffers around stations
station_buf = gdf_stations_proj.geometry.buffer(threshold_meters)
combined_buffer = (
    station_buf.union_all()
)  # turns into single geometry which helps with efficiency
is_within = gdf_joined_proj.geometry.within(combined_buffer)
joined["nearStation"] = is_within.values
joined_post_filter = joined[~joined["nearStation"]]
print("\nDone! \nFaults within 1km of service station labeled in 'joined'.")
print(
    f"When filtered, this removes {len(joined_pre_station_filter['RecordID']) - len(joined_post_filter['RecordID'])} rows"
)
# filter out active=False
joined_active = joined[joined["active"]]
joined = joined_active
print(
    f"\nNumber of rows after filtering active=False out: {len(joined_active['active'])}"
)
print(
    f"Rows removed: {len(joined_pre_station_filter['RecordID']) - len(joined_active['active'])}"
)

Labeling faults near service stations...

Done! 
Faults within 1km of service station labeled in 'joined'.
When filtered, this removes 129284 rows

Number of rows after filtering active=False out: 608454
Rows removed: 578881


In [6]:
target_spn = 5246

# Ensure EventTimeStamp is datetime
joined["EventTimeStamp"] = pd.to_datetime(joined["EventTimeStamp"])

# --- SORTING STEP ---
# Sort by EquipmentID and then chronologically by EventTimeStamp
print("Sorting data by EquipmentID and EventTimeStamp...")
joined = joined.sort_values(by=["EquipmentID", "EventTimeStamp"]).copy()
print("Sorting complete.")

# Create a Series containing only the timestamps of trigger events
trigger_timestamps_only = joined["EventTimeStamp"].where(joined["spn"] == target_spn)

# For each row, find the timestamp of the *next* trigger event within its group
# Group by EquipmentID and use backward fill (bfill)
# This fills NaT values with the next valid timestamp in the group
print("Calculating next trigger time...")
joined["next_trigger_time"] = trigger_timestamps_only.groupby(
    joined["EquipmentID"]
).bfill()

# Calculate the start of the 2-hour window before the next trigger
joined["window_start_time"] = joined["next_trigger_time"] - pd.Timedelta(hours=2.5)

# Label rows as True if their timestamp falls within the window:
#    [wind`ow_start_time, next_trigger_time]
#    Also ensure that a next_trigger_time actually exists (it's not NaT)
print("Labeling derate window...")
joined["derate_window"] = (
    (joined["EventTimeStamp"] >= joined["window_start_time"])
    & (joined["EventTimeStamp"] <= joined["next_trigger_time"])
    & (joined["next_trigger_time"].notna())
)

# --- Verification ---
print("\nVerification:")
print(
    "Value counts for 'derate_window':\n",
    joined["derate_window"].value_counts(),
)
print(
    "\nValue counts for 'spn' (to confirm target SPN exists):\n",
    joined["spn"].value_counts(),
)

# Display some rows where derate_window is True (if any)
print("\nSample rows where derate_window is True:")
print(
    joined[joined["derate_window"]][
        ["EquipmentID", "EventTimeStamp", "spn", "next_trigger_time"]
    ].head()
)

# Display some rows around a trigger event for a specific EquipmentID
# Find an EquipmentID that has a trigger event
example_eqid = joined.loc[joined["spn"] == target_spn, "EquipmentID"].iloc[0]
print(f"\nSample data around trigger for EquipmentID: {example_eqid}")
example_trigger_time = joined.loc[
    (joined["EquipmentID"] == example_eqid) & (joined["spn"] == target_spn),
    "EventTimeStamp",
].min()
# Filter data around that time for that equipment
print(
    joined[
        (joined["EquipmentID"] == example_eqid)
        & (joined["EventTimeStamp"] >= (example_trigger_time - pd.Timedelta(hours=3)))
        & (joined["EventTimeStamp"] <= (example_trigger_time + pd.Timedelta(hours=1)))
    ][
        [
            "EquipmentID",
            "EventTimeStamp",
            "spn",
            "next_trigger_time",
            "window_start_time",
            "derate_window",
        ]
    ]
)

joined = joined.drop(columns=['next_trigger_time', 'window_start_time'])


Sorting data by EquipmentID and EventTimeStamp...


/var/folders/1c/yvl84p5d2rv0n8dkqqjgmkdm0000gn/T/ipykernel_37541/1950150103.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined["EventTimeStamp"] = pd.to_datetime(joined["EventTimeStamp"])


Sorting complete.
Calculating next trigger time...
Labeling derate window...

Verification:
Value counts for 'derate_window':
 derate_window
False    607189
True       1265
Name: count, dtype: int64

Value counts for 'spn' (to confirm target SPN exists):
 spn
111      183752
929      128777
96        48062
829       46179
639       20640
          ...  
703           1
707           1
5395          1
32000         1
1265          1
Name: count, Length: 450, dtype: int64

Sample rows where derate_window is True:
       EquipmentID      EventTimeStamp   spn   next_trigger_time
996835   105349576 2018-07-06 09:42:48  5246 2018-07-06 09:42:48
156408   105360462 2015-08-10 10:26:40  1569 2015-08-10 10:26:40
156409   105360462 2015-08-10 10:26:40  3360 2015-08-10 10:26:40
156410   105360462 2015-08-10 10:26:40  5246 2015-08-10 10:26:40
185581   105411041 2015-08-31 13:50:42  5246 2015-08-31 13:50:42

Sample data around trigger for EquipmentID: 105349576
       EquipmentID      EventTimeStamp

In [7]:
# some feature engineering:
joined['time_since_last_fault'] = joined.groupby('EquipmentID')['EventTimeStamp'].diff()
joined['fault_frequency'] = joined.groupby('EquipmentID')['spn'].transform('count')
time_cols = joined.select_dtypes(include=['datetime64'])
time_cols = list(time_cols.columns)

In [8]:
display(joined)

,RecordID,ESS_Id,EventTimeStamp,ecuSerialNumber,spn,fmi,active,EquipmentID,Latitude,Longitude,...,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,nearStation,derate_window,time_since_last_fault,fault_frequency
1008925,1046062,58447476,2018-08-15 09:55:05,unknown,639,2,True,0105309016,36.194953,-83.175046,...,False,NaN,4.7865,NaN,100,0.87,True,False,NaT,1
985277,1019516,50580811,2011-01-01 01:09:24,80055028,191,9,True,0105319027,35.588333,-86.443796,...,True,NaN,0,NaN,100,0,True,False,NaT,8
985237,1019476,50568634,2011-01-01 02:03:05,80055028,191,9,True,0105319027,35.588333,-86.443796,...,True,NaN,0,NaN,100,0,True,False,0 days 00:53:41,8
985170,1019409,50541505,2011-01-01 03:36:42,80055028,111,18,True,0105319027,35.588240,-86.443796,...,True,NaN,0,NaN,100,0,True,False,0 days 01:33:37,8
985179,1019418,50547462,2018-05-31 07:31:35,80055028,84,9,True,0105319027,35.588148,-86.443750,...,True,NaN,0,NaN,100,0,True,False,2707 days 03:54:53,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157136,1214360,109664481,2019-11-12 00:24:39,S211411637,639,2,True,310,35.271666,-86.397361,...,False,NaN,4.466105,NaN,100,2.61,False,False,0 days 00:07:58,593
4245,4246,1048627,2015-02-24 13:45:06,79751302,1761,17,True,R1762,41.254166,-85.088888,...,False,NaN,65.96243,3276.75,47.2,16.82,False,False,NaT,3
4427,4428,1051023,2015-02-24 15:31:17,79751302,1761,18,True,R1762,39.944444,-86.016990,...,False,NaN,66.83624,3276.75,67.2,8.99,False,False,0 days 01:46:11,3
6438,6439,1089561,2015-02-26 13:12:11,79751302,5848,9,True,R1762,39.952870,-81.936990,...,False,NaN,2.058292,3276.75,0,0.29,False,False,1 days 21:40:54,3
